In [ ]:
import numpy as np
import pandas as pd
from subprocess import check_output
#print(check_output(["ls", "./humanclassifier"]).decode("utf8"))
print(check_output(["ls", "./input"]).decode("utf8"))

In [ ]:
from subprocess import check_output
print(check_output(["ls", "./cropped_input/additional"]).decode("utf8"))
#print(check_output(["ls", "./input/train"]).decode("utf8"))

In [ ]:
import glob
train= glob.glob("./all_images_300/train/**/*.jpg") #+ glob.glob("./all_images_300/additional/**/*.jpg")
#train= glob.glob("./input/train/**/*.jpg")#+glob.glob("./input/additional/**/*.jpg")
train = pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train], columns = ['type','image','path'])
test = glob.glob("./all_images_300/test/*.jpg") + glob.glob("./all_images_300/test_stg2/*.jpg")
#test = glob.glob("./input/test/*.jpg")
test = pd.DataFrame([[p.split('/')[3],p] for p in test], columns = ['image','path'])

In [ ]:
import glob
train = glob.glob("./all_images_300/train/**/*.jpg")
train = pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train], columns = ['type','image','path'])
test = glob.glob("./humanclassifier/All_categories/*.jpg")
test = pd.DataFrame([[p.split('/')[3],p] for p in test], columns = ['image','path'])

In [ ]:
train[train.image.isin(test.image)].type.values

In [ ]:
#csv file containing similar images between additional and test set
#create a list of the additional dataset images to remove from training set
import csv
to_remove = []
with open('./input/test_duplicates_v3.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    next(csvreader)
    for row in csvreader:
        aux = row[1].split('/')
        to_remove.append(aux[-1])
unique_values = set(to_remove)
to_remove_unique = list(unique_values)
print(len(to_remove_unique))

In [ ]:
#clean training set
train = train[~train.image.isin(to_remove_unique)]

In [ ]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import cv2

types = train.groupby('type', as_index=False).count()
types.plot(kind='bar', x='type', y='path', figsize=(7,4))

In [ ]:
types

In [ ]:
from multiprocessing import Pool, cpu_count

def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

train = im_stats(train)
sizes = train.groupby('size', as_index=False)['path'].count()
_ = sizes.plot(kind='bar', x='size', y='path', figsize=(7,4))

In [ ]:
sizes

In [ ]:
# Test bed for first architecture

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.layers.normalization import BatchNormalization

# from keras import backend as K
# K.set_image_dim_ordering('th')

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (200, 200), cv2.INTER_LINEAR)
    return [path, resized]
    #return [path, img]

def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

#train = glob.glob('../input/train/**/*.jpg') + glob.glob('../input/additional/**/*.jpg')
print(len(train))
#train = pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train], columns = ['type','image','path'])
#train = train[train['size'] != '0 0'].reset_index(drop=True) #remove bad images
train_data = normalize_image_features(train['path'])
#np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)
#train_data = np.load('train.npy')

le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
#np.save('train_target.npy', train_target, allow_pickle=True, fix_imports=True)
#train_target = np.load('train_target.npy')

def create_model(opt_):
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(3, 200, 200)))
    model.add(Convolution2D(8, 3, 3))
    model.add(Dropout(0.2))
    model.add(Dense(12))
    model.add(Dropout(0.2))
    model.add(Dense(6))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax'))
#     model = Sequential()
#     model.add(ZeroPadding2D((1, 1), input_shape=(3, 200, 200)))
#     model.add(Convolution2D(32, 3, 3))
#     model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

#     model.add(Conv2D(32, 3, 3))
#     model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

#     model.add(Conv2D(64, 3, 3))
#     model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
    
#     model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
#     model.add(Dense(64))
#     model.add(Dropout(0.5))
#     model.add(Dense(3, activation='softmax'))

    model.compile(optimizer=opt_, loss='sparse_categorical_crossentropy', metrics=['accuracy']) #loss='binary_crossentropy' not working
    return model

model = KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=15, verbose=2)
#model = KerasClassifier(build_fn=create_model, verbose=2)
opts_ = ['adamax'] #['adadelta','sgd','adagrad','adam','adamax'] #['adamax']
epochs = np.array([100])
#epochs = [10, 50, 100]
batches = np.array([15])
#batches = [5, 10, 15, 20]
param_grid = dict(nb_epoch=epochs, batch_size=batches, opt_=opts_)
grid = GridSearchCV(estimator=model, cv=StratifiedKFold(n_splits=2), param_grid=param_grid, verbose=20)
grid_result = grid.fit(train_data, train_target)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
#print("Log Loss...", log_loss(train_target, grid_result.predict(train_data)))

test_data = normalize_image_features(test['path'])
#np.save('test.npy', test_data, allow_pickle=True, fix_imports=True)
#test_data = np.load('test.npy')
test_id = test.image.values
#np.save('test_id.npy', test_id, allow_pickle=True, fix_imports=True)
#test_id = np.load('test_id.npy')

pred = grid_result.predict_proba(test_data)
df = pd.DataFrame(pred, columns=le.classes_)
df['image_name'] = test_id
df.to_csv('submission2.csv', index=False)


In [ ]:
#Starting test bed for final architecture with k-fold cross validation
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    return [path, resized]
    #return [path, img]

def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

#train = glob.glob('../input/train/**/*.jpg') + glob.glob('../input/additional/**/*.jpg')
print(len(train))
#train = pd.DataFrame([[p.split('/')[3],p.split('/')[4],p] for p in train], columns = ['type','image','path'])
#train = train[train['size'] != '0 0'].reset_index(drop=True) #remove bad images
train_data = normalize_image_features(train['path'])
#np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)
#train_data = np.load('train.npy')

le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
#np.save('train_target.npy', train_target, allow_pickle=True, fix_imports=True)
#train_target = np.load('train_target.npy')

In [ ]:
#Final architecure
from keras.layers.convolutional import Conv2D
from keras.layers import Activation, BatchNormalization
from keras import backend as K
K.set_image_dim_ordering('th')

def create_model(opt_):
    model = Sequential()
    model.add(Convolution2D(4, 3, 3, activation='relu', dim_ordering='th', input_shape=(3, 64, 64))) #use input_shape=(3, 64, 64)
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Convolution2D(8, 3, 3, activation='relu', dim_ordering='th'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(16, activation='tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer=opt_, loss='sparse_categorical_crossentropy', metrics=['accuracy']) #loss='binary_crossentropy' not working
    return model

model = KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=15, verbose=2)
#model = KerasClassifier(build_fn=create_model, nb_epoch=100, verbose=2)
opts_ = ['adagrad']#['adadelta','sgd','adagrad','adam','adamax'] #['adamax']
epochs = np.array([100])
#epochs = [10, 50, 100]
batches = np.array([15])
#batches = [5, 10, 15, 20]
param_grid = dict(nb_epoch=epochs, batch_size=batches, opt_=opts_)
grid = GridSearchCV(estimator=model, cv=StratifiedKFold(n_splits=2), param_grid=param_grid, verbose=20)
grid_result = grid.fit(train_data, train_target)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
print("Log Loss...", log_loss(train_target, grid_result.predict_proba(train_data)))

test_data = normalize_image_features(test['path'])
#np.save('test.npy', test_data, allow_pickle=True, fix_imports=True)
#test_data = np.load('test.npy')
test_id = test.image.values
#np.save('test_id.npy', test_id, allow_pickle=True, fix_imports=True)
#test_id = np.load('test_id.npy')

pred = grid_result.predict_proba(test_data)
df = pd.DataFrame(pred, columns=le.classes_)
df['image_name'] = test_id
df.to_csv('human_accuracy_test.csv', index=False)

In [ ]:
test_target = le.fit_transform(train[train.image.isin(test.image)].type.values)
print("Log Loss...", log_loss(test_target, pred))

In [ ]:
# list all data in history
from sklearn_evaluation import plot
plot.grid_search(grid.grid_scores_, change='nb_epoch', kind='bar')